# Chapter 7. Identifying Similar Groups in Data

In many applications, we want to cluster observations into similar groups based on observed features. For example, in marketing, it is common to divide potential customers into groups for the purposes of targeting. (For example, a business might want to target a campaign at a group that is most likely to respond to the campaign.) This practice is known as **market segmentation**.

This chapter is about how to automatically identify clusters in data, a problem known as **clustering**. Like the classification problems we examined in the previous chapter, clustering is about dividing observations into categories based on features. Unlike classification, however, we do not have ground truth labels that specify what the categories should be; we have to infer them automatically from data. In other words, with classification, we have both features $X$ and labels $y$; with clustering, we only have the features $X$.

For this reason, clustering is an example of an **unsupervised learning** problem, in contrast to the **supervised learning** problems of the previous chapters. This unsupervised / supervised terminology comes from the following analogy.  Imagine a child that is trying to learn the difference between shapes and has several examples of each shape in front of him.

<img src="shape_sorter.jpg" />

The child may be _supervised_ by an adult who gives the child feedback on each answer: "Yes, that is a circle....No, that was a square....No, that was a circle."  This process is analogous to classification, where the labels in the training data can be used to evaluate how good the model's predictions are.

On the other hand, the child may be _unsupervised_ and completely left to his own devices. Eventually, he may figure out that there is something similar about all of the circles that distinguish them from the squares. But he won't know that they are called "circles", nor does he have any way of evaluating whether he is right or not. This is the fundamental challenge of clustering.

We will practice clustering on a dataset containing measurements of 150 iris flowers, collected by the statistician R. A. Fisher.

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
pd.options.display.max_rows = 5

iris = pd.read_csv("/data301/data/iris.csv")
iris

Let's focus on just two of the variables: the petal length and width. The reason for choosing just two variables is so that we can easily visualize the data. Based on the scatterplot below, how many clusters do you think there are in this data set? Can you invent an algorithm that would automatically identify those clusters?

In [ ]:
X_train = iris[["petal length", "petal width"]]
X_train.plot.scatter(x="petal length", y="petal width", 
                     c="black", marker="x", alpha=.5)

# Chapter 7.1 $K$-Means Clustering

$K$-means is an algorithm for finding clusters in data. The idea behind $k$-means is simple: each cluster has a "center" point called the **centroid**, and each observation is associated with the cluster of its closest centroid. The only challenge is finding those centroids. The $k$-means algorithm starts with a random guess for the centroids and iteratively improves them.

The steps are as follows:

1. Initialize $k$ centroids at random.
2. Assign each point to the cluster of its nearest centroid.
3. (After the reassignment, the centroid may no longer be at the center of its cluster.) Recompute each centroid based on the points assigned to its cluster.
4. Repeat steps 2 and 3 until no points change clusters.

Let's see how this works in code. First, let's sample 3 points from the data at random to serve as the initial centroids.

In [ ]:
# Initialize 3 centroids at random from the data.
centroids = X_train.sample(3)

# Call the three clusters "red", "blue", "yellow" for convenience.
centroids.index = ["r", "b", "y"]

# Let's plot these centroids.
ax = X_train.plot.scatter(x="petal length", y="petal width", 
                          c="black", marker="x", alpha=.5)
centroids.plot.scatter(x="petal length", y="petal width", 
                       c=centroids.index, ax=ax)

centroids

Now we assign each point to the cluster of its nearest centroid.

In [ ]:
# First, define a function that finds the closest centroid to 
# a given observation.
def get_closest_centroid(obs):
    dists = np.sqrt(((obs - centroids) ** 2).sum(axis=1))
    return dists.idxmin()

get_closest_centroid(X_train.loc[0])

In [ ]:
# Apply the function to the entire data set.
clusters = X_train.apply(get_closest_centroid, axis=1)

# Plot the cluster assignments.
ax = X_train.plot.scatter(x="petal length", y="petal width", 
                          c=clusters, marker="x", alpha=.5)
centroids.plot.scatter(x="petal length", y="petal width", 
                       c=centroids.index, ax=ax)

Notice that some of the centroids are no longer at the center of their clusters. We can fix that by redefining the centroid to be the mean of the points in its cluster.

In [ ]:
# Calculate the mean length and width for each cluster.
centroids = X_train.groupby(clusters).mean()

# Let's plot the new centroids.
ax = X_train.plot.scatter(x="petal length", y="petal width", 
                          c=clusters, marker="x", alpha=.5)
centroids.plot.scatter(x="petal length", y="petal width", 
                       c=centroids.index, ax=ax)

centroids

Now, there may be some points that are no longer assigned to their closest centroid, so we have to go back and re-assign clusters. But that may cause the centroids to no longer be at the center of their cluster, so we have to recalculate the centroids. And so on. This process continues until the cluster assignments stop changing.

In [ ]:
# Assign points to their closest centroid.
clusters = X_train.apply(get_closest_centroid, axis=1)

# Recalculate the centroids based on the clusters.
centroids = X_train.groupby(clusters).mean()

# Plot.
ax = X_train.plot.scatter(x="petal length", y="petal width", 
                          c=clusters, marker="x", alpha=.5)
centroids.plot.scatter(x="petal length", y="petal width", 
                       c=centroids.index, ax=ax)

We can run the above code over and over until the clusters stop changing. This is the final cluster assignment.

It is not so easy to visualize the clusters when there are more than 2 features. But we can wrap the same algorithm inside a loop that continues until the cluster assignments do not change from one step to the next. One of the optional exercises below walks you through such an implementation.

However, we rarely need to implement $k$-means from scratch. This is because $k$-means is implemented in Scikit-Learn. The API for Scikit-Learn's $k$-means model is similar to the API for supervised learning models, like $k$-nearest neighbors, except that the `.fit()` method only takes in `X`, not `X` and `y`. This makes sense because in unsupervised learning, there are no ground truth labels `y`.

In [ ]:
from sklearn.cluster import KMeans

model = KMeans(n_clusters=3)
model.fit(X_train)

In [ ]:
# Extract the centroids and the clusters.
centroids = model.cluster_centers_
clusters = model.labels_

clusters

In [ ]:
# Map the cluster numbers to colors.
clusters = pd.Series(clusters).map({
    0: "r",
    1: "b",
    2: "y"
})

# Plot the data
X_train.plot.scatter(x="petal length", y="petal width", 
                     c=clusters, marker="x", alpha=.5)

We can call `model.predict()` to get the cluster assignment for a new observation. This will simply assign the new observation to the nearest cluster without recalculating the centroids. (If this observation had been in the training data, then assigning the new observation to a cluster would require recalculating the centroid, which would in turn require reassigning observations to clusters, and so on.)

For example, consider a flower whose petal has a length of 5.0 and a width of 0.5. It's obvious which cluster this point should be assigned to. Let's check that this is indeed the case, by calling `.predict()` on our fitted model.

In [ ]:
model.predict([[5.0, 0.5]])

### A Note about Standardization

Note that we did not standardize the variables in the example above. Just as with $k$-nearest neighbors, it is generally a good idea to standardize your variables before applying $k$-means. Let's see how different the clusters would be if the variables had been standardized.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)

model = KMeans(n_clusters=3)
model.fit(X_train_std)

clusters = pd.Series(model.labels_).map({
    0: "r",
    1: "b",
    2: "y"
})

X_train.plot.scatter(x="petal length", y="petal width", 
                     c=clusters, marker="x", alpha=.5)

Let's compare these clusters to the ones we obtained above, keeping in mind that the coloring of the clusters is arbitrary. (All that matters is which observations are grouped together.) We see that only a handful observations near the boundary actually change cluster.  Standardization turns out not to matter much for the iris dataset because the variables are on similar scales. But in data sets with variables on different scales, standardization is crucial.

# Exercises

**Exercise 1.** Use $k$-means to cluster the wines in the wines dataset into 2 clusters. (Code to read in this dataset has been provided below.) How well do your two clusters correspond to white and red wines? (The way the dataset is read in below, the first 1599 wines are red, and the rest are white.)

_Hint:_ Don't forget to standardize your variables first!

In [ ]:
# TYPE YOUR CODE HERE.
red = pd.read_csv("/data301/data/winequality-red.csv", sep=";")
white = pd.read_csv("/data301/data/winequality-white.csv", sep=";")
wines = pd.concat([red, white], ignore_index=True)


**Exercise 2.** Use $k$-means to cluster the Titanic passengers (`/data301/data/titanic.csv`) into $k$ clusters. You are free to choose the number of clusters $k$ and the features to include (but be sure to include both categorical and quantitative features). Look at the profiles of the passengers in each cluster. Can you come up with an "interpretation" of each cluster based on the passengers in it?

In [ ]:
# TYPE YOUR CODE HERE.

**Exercise 3.** The code below reads in the "two moons" dataset, a synthetic dataset that is used to evaluate clustering algorithms. What should be the two clusters be _intuitively_? What do you think $k$-means will return as the clusters? Once you have a hypothesis, test it out by fitting the model to this dataset and plotting the resulting clusters.

In [ ]:
# TYPE YOUR CODE HERE
moons = pd.read_csv("/data301/data/two_moons.csv")
moons.plot.scatter(x="x1", y="x2", color="k")

**Exercise 4.** The code below reads in the "satellite" dataset, a synthetic dataset that is used to evaluate clustering algorithms. What should the two clusters be _intuitively_? What will the clusters be if you ask $k$-means to cluster this data into 2 clusters? Once you have a hypothesis, test it out by running $k$-means on this dataset.

In [ ]:
# TYPE YOUR CODE HERE
satellite = pd.read_csv("/data301/data/satellite.csv")
satellite.plot.scatter(x="x1", y="x2", color="k")

**Challenge Exercise.** Write a function, `get_kmeans_clusters`, that takes in a `DataFrame` and a number of clusters, and returns a `Series` containing the cluster assignment of each observation (row).

In [ ]:
def get_kmeans_clusters(X_train, k):
    raise NotImplementedError